In [291]:
from functools import reduce
from typing import Tuple, Dict, Any
import pandas as pd
import numpy as np
import altair as alt

In [292]:
# The SIR model, one time step
def sir(y, beta, gamma, N):
    S, I, R = y
    Sn = (-beta * S * I) + S
    In = (beta * S * I - gamma * I) + I
    Rn = gamma * I + R
    if Sn < 0:
        Sn = 0
    if In < 0:
        In = 0
    if Rn < 0:
        Rn = 0

    scale = N / (Sn + In + Rn)
    return Sn * scale, In * scale, Rn * scale

In [293]:
# Run the SIR model forward in time
def sim_sir(S, I, R, beta, gamma, n_days, beta_decay=None):
    N = S + I + R
    s, i, r = [S], [I], [R]
    for day in range(n_days):
        y = S, I, R
        S, I, R = sir(y, beta, gamma, N)
        if beta_decay:
            beta = beta * (1 - beta_decay)
        s.append(S)
        i.append(I)
        r.append(R)

    s, i, r = np.array(s), np.array(i), np.array(r)
    return s, i, r

In [294]:
def projected_admissions(data_dict):
    """Projected number of new COIVD patients in hospital per day"""

    projection = pd.DataFrame.from_dict(data_dict)
    projection_admits = projection.iloc[:-1, :] - projection.shift(1)
    projection_admits[projection_admits < 0] = 0

    plot_projection_days = n_days - 10
    projection_admits["day"] = range(projection_admits.shape[0])

    return projection_admits

project_admits = projected_admissions(data_dict)

In [295]:
def _census_table(projection_admits, hosp_los, icu_los, vent_los) -> pd.DataFrame:
    """Projected number of total COIVD patients in hospital per day"""

    los_dict = {
        "hosp": hosp_los,
        "icu": icu_los,
        "vent": vent_los,
    }

    census_dict = dict()
    for k, los in los_dict.items():
        census = (
            projection_admits.cumsum().iloc[:-los, :]
            - projection_admits.cumsum().shift(los).fillna(0)
        ).apply(np.ceil)
        census_dict[k] = census[k]


    census_df = pd.DataFrame(census_dict)
    census_df["day"] = census_df.index
    census_df = census_df[["day", "hosp", "icu", "vent"]]

    census_table = census_df[np.mod(census_df.index, 1) == 0].copy()
    census_table.index = range(census_table.shape[0])
    census_table.loc[0, :] = 0
    census_table = census_table.dropna().astype(int)

    return census_table

census_table = _census_table(project_admits, hosp_los, icu_los, vent_los)

In [305]:
recs = pd.read_csv('COVID_Sample_2019.csv').to_dict('records')

In [311]:
def get_counts(record):
    
    current_hosp = record['PATIENTS']
    S = record['POPULATION']
    initial_infections = record['COUNTY_CASES']
    market_share = record['MARKET_SHARE']

    #current_hosp = 4

    doubling_time = 6

    relative_contact_rate = 0

    hosp_rate = 0.05

    icu_rate = 0.02

    vent_rate = 0.01

    hosp_los = 7

    icu_los = 9

    vent_los = 10

    #market_share = 0.2

    #S = 4119405

    #initial_infections = 91

    recovery_days = 14.0

    n_days = 60

    beta_decay = 0.0
    
    #initial calculations
    total_infections = current_hosp / market_share / hosp_rate
    detection_prob = initial_infections / total_infections

    S, I, R = S, initial_infections / detection_prob, 0

    intrinsic_growth_rate = 2 ** (1 / doubling_time) - 1

    # mean recovery rate, gamma, (in 1/days).
    gamma = 1 / recovery_days

    # Contact rate, beta
    beta = (
        intrinsic_growth_rate + gamma
    ) / S * (1-relative_contact_rate) # {rate based on doubling time} / {initial S}

    r_t = beta / gamma * S # r_t is r_0 after distancing
    r_naught = r_t / (1-relative_contact_rate)
    doubling_time_t = 1/np.log2(beta*S - gamma +1) # doubling time after distancing

    s, i, r = sim_sir(S, I, R, beta, gamma, n_days, beta_decay=beta_decay)

    hosp = i * hosp_rate * market_share
    icu = i * icu_rate * market_share
    vent = i * vent_rate * market_share

    days = np.array(range(0, n_days + 1))
    data_list = [days, hosp, icu, vent]
    data_dict = dict(zip(["day", "hosp", "icu", "vent"], data_list))
    
    project_admits = projected_admissions(data_dict)
    
    census_table = _census_table(project_admits, hosp_los, icu_los, vent_los)
    
    daily_admits = projected_admissions(data_dict).to_dict('records')[30]
    census = census_table.to_dict('records')[30]
    
    census['HospitalizedDay30'] = census.pop('hosp')
    census['ICUDay30'] = census.pop('icu')
    census['VentDay30'] = census.pop('vent')
    del census['day']
    daily_admits['HospitalizedDailyDay30'] = daily_admits.pop('hosp')
    daily_admits['ICUDailyDay30'] = daily_admits.pop('icu')
    daily_admits['VentDailyDay30'] = daily_admits.pop('vent')
    del daily_admits['day']
    
    record.update(census)
    record.update(daily_admits)

    overflow_risk = record['HospitalizedDay30'] / record['TOTAL_HOSPITAL_BEDS']

    record.update({"OverflowRisk" : overflow_risk})
    
    return record

In [313]:
final_data = []
for rec in recs:
    try:
        final_data.append(get_counts(rec))
    except:
        pass

In [315]:
pd.DataFrame(final_data).to_csv('Overflow_Risks.csv')

In [314]:
rec

{'POPULATION': 23081,
 'COUNTYFIPS': 56039,
 'LATITUDE': 43.48028185,
 'LONGITUDE': -110.7495779,
 'NAME': 'TETON COUNTY HOSPITAL DISTRICT',
 'ADDRESS': '625 EAST BROADWAY',
 'CITY': 'JACKSON',
 'STATE': 'WY',
 'COUNTY': 'TETON',
 'ZIP': 83001,
 'TELEPHONE': '(307) 733-3636',
 'TYPE': 'GENERAL ACUTE CARE',
 'STATUS': 'OPEN',
 'SOURCE': 'https://health.wyo.gov/aging/hls/healthcare-facility-directory/',
 'WEBSITE': 'http://www.tetonhospital.org/',
 'OWNER': 'NON-PROFIT',
 'TOTAL_BEDS_IN_COUNTY': 48.0,
 'MARKET_SHARE': 1.0,
 'PROVIDERNUMBER': nan,
 'TOTAL_HOSPITAL_BEDS': 48.0,
 'ICU_HOSPITAL_BEDS': nan,
 'CONFIRMED_CASES_STATE': 15.0,
 'DEATHS_STATE': 0.0,
 'RECOVERED_STATE': 0.0,
 'COUNTY_CASES': 1.0,
 'PATIENTS': 0.2,
 'HospitalizedDay30': 4,
 'ICUDay30': 2,
 'VentDay30': 1,
 'HospitalizedDailyDay30': 0.683618763163441,
 'ICUDailyDay30': 0.2734475052653762,
 'VentDailyDay30': 0.1367237526326881,
 'OverflowRisk': 0.08333333333333333}